In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

# --------------------------------------------------
# PASO 1: Cargar el DataFrame y convertir 'date_time' a datetime
# --------------------------------------------------
df = pd.read_csv("../../data/processed/navegacion_clientes_experimento_limpio.csv")
df["date_time"] = pd.to_datetime(df["date_time"])

# --------------------------------------------------
# PASO 2: Filtrar registros relevantes: 'start' y 'confirm'
# --------------------------------------------------
df_times = df[df["process_step"].isin(["start", "confirm"])].copy()

# --------------------------------------------------
# PASO 3: Agrupar a nivel de sesión (visit_id y variation)
# --------------------------------------------------
# Para cada sesión, se obtiene:
# - start_time: la fecha/hora mínima cuando se registró 'start'
# - confirm_time: la fecha/hora máxima cuando se registró 'confirm'
def get_start(x):
    # Filtrar valores donde process_step es "start"
    return x[df_times.loc[x.index, "process_step"] == "start"].min()

def get_confirm(x):
    # Filtrar valores donde process_step es "confirm"
    return x[df_times.loc[x.index, "process_step"] == "confirm"].max()

df_session_times = (
    df_times.groupby(["visit_id", "variation"], as_index=False)
            .agg(start_time=("date_time", get_start),
                 confirm_time=("date_time", get_confirm))
)

# Eliminar sesiones sin confirm (donde confirm_time es NaT)
df_session_times = df_session_times.dropna(subset=["confirm_time"])

# --------------------------------------------------
# PASO 4: Calcular Time to Complete (TTC)
# --------------------------------------------------
# Se calcula el tiempo de finalización en minutos para cada sesión:
df_session_times["TTC_minutes"] = (df_session_times["confirm_time"] - df_session_times["start_time"]).dt.total_seconds() / 60

# --------------------------------------------------
# PASO 5: Separar los grupos Test y Control
# --------------------------------------------------
ttc_test = df_session_times[df_session_times["variation"] == "Test"]["TTC_minutes"]
ttc_control = df_session_times[df_session_times["variation"] == "Control"]["TTC_minutes"]

# --------------------------------------------------
# PASO 6: Formular la hipótesis y realizar el test
# --------------------------------------------------
# Hipótesis:
# H0: La media de TTC en Test es igual a la media de TTC en Control.
# H1: La media de TTC en Test es menor que la de Control (ya que menor TTC implica mejor UX).

# Se usa un test t de muestras independientes (suponiendo varianzas desiguales)
t_stat, p_value_two_tailed = stats.ttest_ind(ttc_test, ttc_control, equal_var=False)

# Convertir a p-value one-tailed (para H1: Test < Control)
if t_stat < 0:
    p_value_one_tailed = p_value_two_tailed / 2
else:
    p_value_one_tailed = 1 - p_value_two_tailed / 2

print("=== Time to Complete (TTC) por sesión (en minutos) ===")
print(f"Media TTC Test   : {ttc_test.mean():.2f} minutos")
print(f"Media TTC Control: {ttc_control.mean():.2f} minutos")
print(f"t-statistic      : {t_stat:.4f}")
print(f"p-value (one-tailed): {p_value_one_tailed:.6f}")

alpha = 0.05
if p_value_one_tailed < alpha:
    print(f"\nConclusión: Rechazamos H0. El grupo Test tiene un TTC significativamente menor que Control (p < {alpha}).")
else:
    print(f"\nConclusión: No rechazamos H0. No se evidencia que el grupo Test tenga un TTC menor que Control (p >= {alpha}).")


=== Time to Complete (TTC) por sesión (en minutos) ===
Media TTC Test   : 6.93 minutos
Media TTC Control: 6.73 minutos
t-statistic      : nan
p-value (one-tailed): nan

Conclusión: No rechazamos H0. No se evidencia que el grupo Test tenga un TTC menor que Control (p >= 0.05).
